In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
def str_reais_para_float(valor):
    return float(valor.replace("R$ ", "").replace(".", "").replace(",", "."))

In [6]:
def ler_pagina_salarios(response):
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'lxml')
        tables = soup.find("table", {"id": "tabela_principal"})
        trs = tables.find_all("tr")
        
        funcionarios = []

        for tr in trs:
            nome_valor = tr.find("td", {"class": "nome_valor"})
            cargo_valor = tr.find("td", {"class": "cargo_valor"})
            funcao_valor = tr.find("td", {"class": "funcao_valor"})
            remuneracao_valor = tr.find("td", {"class": "remun_valor"})

            nome, cargo, remuneracao_liquida, remuneracao_liquida_pos_desconto = None, None, None, None

            try:
                if nome_valor is not None:
                    nome = nome_valor.string.strip()

                if cargo_valor is not None:
                    cargo = cargo_valor.string.strip()

                if remuneracao_valor is not None:

                    remun_dir = remuneracao_valor.find_all("td", {"class": "remun_dir"})
                    remun_unica = remuneracao_valor.find("td", {"class": "remun_unica"}) 

                    if len(remun_dir) == 0:
                        remuneracao_liquida = str_reais_para_float(remun_unica.a.string.strip())
                    else:
                        remuneracao_liquida = str_reais_para_float(remun_dir[0].a.string.strip())
                        remuneracao_liquida_pos_desconto = str_reais_para_float(remun_dir[1].a.string.strip())

                if nome_valor is not None:
                    funcionario = dict(nome=nome, cargo=cargo, remuneracao_liquida=remuneracao_liquida, 
                                       remuneracao_liquida_pos_desconto=remuneracao_liquida_pos_desconto)
                
                    funcionarios.append(funcionario)
            except Exception as e:
                pass
        
        return funcionarios
        
    else:
        print('Response:', response.status_code)

In [7]:
def buscar_salarios(mes):
    """
        mes: (int)
    """
    url = "http://www2.camara.sp.gov.br/SalariosAbertos/HTML_ativos_2016_{0}/todos.html".format(str(mes).zfill(2))
    response = requests.get(url)
    return ler_pagina_salarios(response)

In [8]:
salarios = buscar_salarios(10)

In [9]:
df = pd.DataFrame().from_records(salarios)

In [10]:
df.sort_values(by='remuneracao_liquida', ascending=False).head(15)

,cargo,nome,remuneracao_liquida,remuneracao_liquida_pos_desconto
1639,PROCURADOR LEGISLATIVO,BRENO GANDELMAN,41151.77,41116.61
1640,PROCURADOR LEGISLATIVO,RAIMUNDO BATISTA,40968.69,38301.74
1837,PROCURADOR LEGISLATIVO,MARCELLA FALBO GIACAGLIA,38519.85,38484.69
1710,TÉCNICO ADMINISTRATIVO,ANGELA BORDIN ANDREONI,36730.72,36013.17
68,PROCURADOR LEGISLATIVO,ANTONIO RODRIGUES DE FREITAS JUNIOR,35297.85,32217.85
1547,TÉCNICO ADMINISTRATIVO,RUI HIROSHI TAMASHIRO,34039.10,33977.44
1859,PROCURADOR LEGISLATIVO,MANOEL JOSE ANIDO FILHO,33581.18,NaN
1874,PROCURADOR LEGISLATIVO,LUIZ EDUARDO DE SIQUEIRA S THIAGO,33243.62,NaN
147,PROCURADOR LEGISLATIVO,KAREN LIMA VIEIRA,33060.44,28570.95
1927,TÉCNICO PARLAMENTAR (PS),JOSE CARLOS TEIXEIRA DE CAMARGO FILHO,32015.41,24135.00
